# TP1 Révisions et courbes de performance
## M2 informatique, Université d'Orléans 2021/2022

Soit une matrice $A$ de taille $n\times n$ et un ensemble de $m$ vecteurs $V$ tel que $V^i$, $0\leq i \leq m-1$ est un vecteur de taille $n$. On souhaite réaliser les $m$ produits matrice vecteur $A\times V^i$. Par contre les vecteurs sont définis tels que les $k$ premiers éléments soient nuls avec $k$ variant pour chaque $i$. Autrement dit la complexité de chaque produit matrice vecteur est différente. 

Dans un premier temps on souhaite utiliser une programmation hybride pour paralléliser cet ensemble de calculs. Cette version va permettre de faire des mesures de performances qui serviront de référence par la suite. 

A partir de l'archive disponible sur Celene, vous disposez des codes suivants 

In [ ]:
// Dans fonctions.cpp (fonctions.h)

void generation_vecteur(int n, int* vecteur, int nb_zero); // Pour générer un vecteur d'entiers de longueur n dont 
                                                           //les nb_zero premiers éléments sont nuls
    
void matrice_vecteur(int n, int* matrice, int* v1, int* v2); // Pour calculer le produit matrice vecteur matrice x v1 = v2
                                                             // La fonction calcule le nombre d'éléments à 0 dans v1
                                                             // et en tient compte pour faire le calcul




Dans **main.cpp**, le code est commenté et doit être complété pour paralléliser le calcul des différents produits matrice vecteur. 

Initialement le processus *root* génére la matrice et l'ensemble des vecteurs (génération aléatoire y compris pour le nombre d'éléments à 0 dans les vecteurs initiaux). Il sauvegarde également les données générés dans un fichier.

### 1. Parallélisation avec MPI

Vous devez suivre les étapes suivantes
1. le processus *root* diffuse la matrice et partage l'ensemble des vecteurs dont on veut calculer le produit avec la matrice.
2. chaque processus effectue ses calculs en utilisant la fonction *matrice_vecteur*
3. Les résultats sont rassemblés sur le processus *root* qui les sauvegardera également dans son fichier.

A noter que le squelette contient également le calcul du temps écoulé de la communication des données à leur rassemblement sur le processus *root*.

Pour rappel l'exécution de fait par la commande ci-dessous et on utilise donc l'environnement d'exécution fourni par MPI. La commande ici correspond à un lancement sur une machine simple.


In [ ]:
mpirun -np 4 ./main 100 10 0 res.txt
// Les arguments sont dans l'ordre
// La taille de la matrice ici 100 x 100
// Le nombre de vecteurs 
// Le processus root 
// Le nom du fichier pour sauvegarder les données.

####  Performances

Q1. Comment définir l'accélération et l'efficacité d'un programme parallèle
Q2. Si vous devez faire des courbes de performance dont l'axe des y est l'axe du temps en secondes quels sont les paramètres que vous pouvez faire varier pour analyser la performance de votre programme ?

Pour faire des mesures de performance, vous allez travailler sur **ptimirev**. La machine **ptimirev** est une grappe de PC reliés par un réseau Ethernet. Elle est constituée de 5 nœuds dont ptimirev-server qui est le point d'entrée à partir de votre compte des salles machines (adresse IP de ptimirev-server 192.168.80.201). Ensuite vous pouvez accéder aux 4 autres nœuds ptimirev1, ptimirev2, ptimirev3 et ptimirev4. Pour travailler sur cette machine vous devez suivre les instructions suivantes 

__Préliminaires__
    1. ssh o'num étudiant'@ptimirev-server (mot de passe code NNE)
    2. ssh-keygen (tout valider sans rentrer de mot de passe)
    3. cat .ssh/id_rsa.pub >> .ssh/authorized_keys (pour ajouter la clé publique aux autorisations)
    4. for i in 1 2 3 4; do ssh ptimirev$i echo Ok; done (valider l'identité des 4 machines)
    5. félicitations ! vous pouvez désormais vous connecter sans mot de passe aux 4 nœuds
    
Si vous préférez utiliser une clé SSH avec mot de passe, alors il est nécessaire avant de lancer MPI d'activer un agent d'authentification SSH et de lui fournir la clé privée. Cela peut se faire, par exemple, en une commande qu'il faudra utiliser à **chaque session** : eval $(ssh-agent); ssh-add

__Gestion de l'exécution parallèle__

Lorsqu'on exécute le programme sur une machine à mémoire distribuée, **mpirun** gère le lancement du programme sur chaque processus à distance par ssh. Il faut donc que l'exécutable soit accessible via le **PATH**. Pour cela vous pouvez rajouter au fichier **.bashrc** les lignes suivantes


In [ ]:
if [ -d "$HOME/bin" ] ; then
    export PATH="$HOME/bin:$PATH"
fi

Ainsi vous pourrez placer votre exécutable dans ce répertoire bin de votre $HOME et lors de l'exécution parallèle le lancement à distance par mpirun pourra fonctionner.

Enfin, avant de pouvoir exécuter le programme il faut définir les processus que l'on souhaite utiliser. Pour cela il faut créer un fichier qui va contenir ces informations. Voici un exemple (le fichier créé se nomme liste_machines)

ptimirev1 slots=2
ptimirev2 slots=2
ptimirev3 slots=2
ptimirev4 slots=2

La syntaxe pour utiliser ce fichier et lancer le programme sur les processus correspondants est

mpirun --hostfile liste_machines (-np 4) 'NomExecutable' 'les arguments'

Si vous précisez le nombre de processus avec l'option -np x le programme s'exécutera sur x processus choisis dans le fichier listes_machines. Sans cette option il utilise toutes les machines données dans le fichier.

Pour avoir des informations sur une machine vous pouvez lire le fichier */proc/cpuinfo* (nombre de processeurs et nombre de coeurs par exemple).

### 2. Parallélisation hybride
Vous avez dû voir que dans **main.cpp** MPI a été initialisé afin de permettre de faire de la programmation hybride et que chaque processus puisse utiliser plusieurs threads pour paralléliser un calcul.

Pour le moment le produit matrice vecteur n'est pas du tout optimisé.

#### Performance